In [27]:
from bs4 import BeautifulSoup
import re
import requests
import json
import lxml

In [28]:
douban_url = "https://book.douban.com/subject/35513147/"
headers = {
    "User-Agent": 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.7 (KHTML, like Gecko) Chrome/20.0.1099.0 Safari/536.7 QQBrowser/6.14.15493.201'
}
html_doc = requests.get(douban_url, headers=headers).text

In [29]:
soup = BeautifulSoup(html_doc, 'lxml')
soup

<!DOCTYPE html>
<html class="ua-windows ua-webkit book-new-nav" lang="zh-cmn-Hans">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>学会提问（原书第12版） (豆瓣)</title>
<script>!function(e){var o=function(o,n,t){var c,i,r=new Date;n=n||30,t=t||"/",r.setTime(r.getTime()+24*n*60*60*1e3),c="; expires="+r.toGMTString();for(i in o)e.cookie=i+"="+o[i]+c+"; path="+t},n=function(o){var n,t,c,i=o+"=",r=e.cookie.split(";");for(t=0,c=r.length;t<c;t++)if(n=r[t].replace(/^\s+|\s+$/g,""),0==n.indexOf(i))return n.substring(i.length,n.length).replace(/\"/g,"");return null},t=e.write,c={"douban.com":1,"douban.fm":1,"google.com":1,"google.cn":1,"googleapis.com":1,"gmaptiles.co.kr":1,"gstatic.com":1,"gstatic.cn":1,"google-analytics.com":1,"googleadservices.com":1},i=function(e,o){var n=new Image;n.onload=function(){},n.src="https://www.douban.com/j/except_report?kind=ra022&reason="+encodeURIComponent(e)+"&environment="+encodeURIComponent(o)},r=function(o){try{t.call(e,o)}catch(e){

In [30]:
schema = soup.find('script', type="application/ld+json")
schema = json.loads(schema.string)
schema

{'@context': 'http://schema.org',
 '@type': 'Book',
 'workExample': [],
 'name': '学会提问（原书第12版）',
 'author': [{'@type': 'Person', 'name': '[美] Neil Browne'},
  {'@type': 'Person', 'name': '[美] Stuart Keeley'}],
 'url': 'https://book.douban.com/subject/35513147/',
 'isbn': '9787111680925',
 'sameAs': 'https://book.douban.com/subject/35513147/'}

In [31]:
title = schema['name'].strip()
title

'学会提问（原书第12版）'

In [32]:
author_list = []
for author in schema['author']:
    name = author['name']
    name = re.sub(r'\[.*\]', '', name)
    name = name.strip()
    author_list.append({
        "name": name
    })
print(author_list)

[{'name': 'Neil Browne'}, {'name': 'Stuart Keeley'}]


In [38]:
soup.find('meta', property="og:image")
cover_url = soup.find('meta', property="og:image")['content']
cover_url

'https://img9.doubanio.com/view/subject/l/public/s33936664.jpg'

In [48]:
publisher_tag = soup.find('span', string="出版社:")
publisher = publisher_tag.next_sibling.next_sibling.string
publisher

'机械工业出版社'

In [109]:
original_name_tag = soup.find('span', string="原作名:")
original_name = original_name_tag.next_sibling
type(original_name)
str(original_name).strip()

'Asking the Right Questions: A Guide to Critical Thinking'

In [61]:
translator_tag = soup.find('span', string=" 译者")
translator_tag = translator_tag.find_next_siblings('a')
translator_list = [{"name": str(translator.string)} for translator in translator_tag]
translator_list

[{'name': '许蔚翰'}, {'name': '吴礼敬'}]

In [65]:
published_tag = soup.find('span', string="出版年:").next_sibling
published = str(published_tag)
if len(published.split('-')) == 2:
    published = published + "-1"
published

' 2021-7-1'

In [75]:
pages_tag = soup.find("span", string="页数:").next_sibling
pages = int(pages_tag.string)
pages

280

In [78]:
douban_ranking = soup.find("strong", property="v:average").string
douban_ranking = float(douban_ranking)
douban_ranking

8.6

In [105]:
intro_tag_list = soup.findAll("div", class_="intro")
if intro_tag_list[0].find('a'):
    intro_tag = intro_tag_list[1]
else:
    intro_tag = intro_tag_list[0]

description = ""
for paragraph in intro_tag.children:
    description = description + paragraph.string
    description = description + '\n'
description.strip()

'如果你所在城市的地铁因为安全因素升级了安检程序，你会积极配合还是消极抵触？如果你喜欢的一位明星或KOL热情地向你推荐一款产品，你会立即下单还是深思熟虑?如果你的老板要求和鼓励你"996"甚至"007"，你会配合还是拒绝？......\n \n我们生活在嘈杂、喧嚣和不确定的世界里，每时每刻都被泛滥的信息所包围，大到世界经济发展的趋势，小到个人生活的决策，面对别人和媒体兜售的观点，如果你缺乏独立思考和判断，就只能全盘接受--他们热衷于让你相信这是"事实"，但所说的话往往真假参半。你明明觉得有什么不对劲，可一时又很难找到突破口反驳。\n \n即使专家说得再动听，也不一定靠谱，我们有必要依靠自己的大脑，不草率、不盲从，不为感性和无事实根据的传闻所左右。本书将帮助我们有意识地学习和运用批判性思维，提出批判性问题，对问题深思熟虑，尽力理解那些价值观和我们背道而驰的人的分析推理方式，克服偏见对判断的影响，这样才有可能得出更为正确、理性的结论。\n \n当你能够坦然面对立场和你完全相左的意见，理性判断而不会党同伐异时，当你面对提问和质疑，有能力组织更多确凿的证据支持自己的观点，而不只把声高当有理，或认为别人是没事找茬，有意和自己过不去时，当你能够在对话和辩论中拨开迷雾，识别出对方的逻辑谬误和情感操纵时，世界在你眼中将变得更加有趣、开阔和坚定。'